# Ligthning Fast One-Shot Side Entanglement Optimization

## Imports

In [611]:
import numpy as np
import random
import itertools
import math
import time
import pickle
from typing import Optional, Tuple, cast, List, Dict
from bitarray.util import count_xor, urandom
from bitarray import frozenbitarray
from qiskit import Aer, QuantumCircuit, execute, QuantumRegister, ClassicalRegister
from qiskit.result.result import Result
from qiskit.aqua.components.optimizers import SLSQP, L_BFGS_B, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES

## General auxiliary functions

In [612]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [613]:
def get_combinations_two_etas_without_repeats_from_etas(angles_etas: List[float]) -> List[Tuple[float, float]]:
    """ from a given list of attenuations factors create a
        list of all combinatorial pairs of possible etas
        without repeats
        For us it is the same testing first eta 0.1 and second eta 0.2
        than first eta 0.2 and second eta 0.1
        Though, we will always put the greater value as the first pair element
    """
    # when there is only one element, we add the same element
    if len(angles_etas) == 1:
        angles_etas.append(angles_etas[0])
    # get combinations of two etas without repeats
    eta_pairs = list(itertools.combinations(angles_etas, 2))

    return reorder_pairs(eta_pairs)

In [614]:
def reorder_pairs(pairs: List[Tuple[float, float]]) -> List[Tuple[float, float]]:
    """ reorder received pairs setting first the element of the tuple
        as greater or equal de second one
    """
    reordered_pairs = pairs
    for idx, pair in enumerate(pairs):
        reordered_pairs[idx] = reorder_pair(pair)
    return reordered_pairs

In [615]:
def reorder_pair(pair: Tuple[float, float]) -> Tuple[float, float]:
    if pair[0] < pair[1]:
        return (pair[1], pair[0])
    return pair

In [616]:
GLOBAL_BACKEND=Aer.get_backend('qasm_simulator')
GLOBAL_ETA_PAIR = (0,0)
optimization_setup = { 'optimizer_algorithms': ['DIRECT_L'],
                        'optimizer_iterations': [1000],
                        'attenuation_angles': np.append(np.arange(0, np.pi/2, np.pi/2/20), np.pi/2),
                        'initial_parameters': [0, 0, 0, 0, 0],
                        'variable_bounds': [(0, 1),   # amplitude_probability
                                            (0, 2*np.pi),  # rx1
                                            (0, 2*np.pi),  # ry1
                                            (0, 2*np.pi),  # rx0
                                            (0, 2*np.pi)], # ry0
                        'plays': 10000}
GLOBAL_ETA_PAIRS = get_combinations_two_etas_without_repeats_from_etas(optimization_setup['attenuation_angles'])
GLOBAL_PLAYS = optimization_setup['plays']
filename='20210412f_C2_A2_1000_10000'

## Auxiliary functions for preparing the optimization (until the cost function)

In [617]:
def find_optimal_configurations(optimization_setup: dict, clone_setup: Optional[dict]=None) -> dict:
    """ Finds out the optimal configuration for each pair of attenuation levels
        using the configured optimization algorithm """
    eta_pairs_idx_to_optimize, optimal_configurations = _select_eta_pairs_to_optimize(clone_setup)

    print(f'number of eta_pairs_idx_to_optimize: {len(eta_pairs_idx_to_optimize)} -> {eta_pairs_idx_to_optimize}')

    program_start_time = time.time()
    print("Starting the execution")
    
    for eta_pair_idx in eta_pairs_idx_to_optimize:
        start_time = time.time()
        GLOBAL_ETA_PAIR = GLOBAL_ETA_PAIRS[eta_pair_idx]
        result = _compute_best_configuration(optimization_setup, GLOBAL_ETA_PAIR)
        optimal_configurations['probabilities'].append(result['best_probability'])
        optimal_configurations['configurations'].append(result['best_configuration'])
        optimal_configurations['best_algorithm'].append(result['best_algorithm'])
        optimal_configurations['number_calls_made'].append(result['number_calls_made'])
        end_time = time.time()
        print(f"Pair of etas # {eta_pair_idx} of {len(eta_pairs_idx_to_optimize)}, time taken this pair of etas: " +
              f'{np.round((end_time - start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - start_time) % 60, 0)} seconds')
        print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time) / 60)))
        print("total time taken so far: " +
              f'{np.round((end_time - program_start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - program_start_time) % 60, 0)} seconds')
    end_time = time.time()
    print("total minutes of execution time: ", int(np.round((end_time - program_start_time) / 60)))
    print(f'Number eta pairs optimized: {len(eta_pairs_idx_to_optimize)}' +
          f'from the total eta pairs: {len(GLOBAL_ETA_PAIRS)} ')
    optimal_configurations['eta_pairs'] = GLOBAL_ETA_PAIRS

    return optimal_configurations

In [618]:
    def _select_eta_pairs_to_optimize(clone_setup: Optional[dict]) -> Tuple[List[int], dict]:
        """ from the given clone setup, select the eta pairs to be optimized,
            and set the non computed pairs configuration as default values   """

        eta_pair_idx_init, eta_pair_idx_end = _set_eta_pair_index_bounds(clone_setup)
        index_dict = _build_eta_pair_index_lists(eta_pair_idx_init, eta_pair_idx_end)
        default_optimal_configurations = _set_default_optimal_configurations(index_dict['eta_pair_idx_to_skip'])

        return (index_dict['eta_pair_idx_to_compute'],
                default_optimal_configurations)

In [619]:
def _set_default_optimal_configurations(eta_pair_idx_to_skip: List[int]) -> dict:
    """ Return the optimal configurations set to default values for the indexes to be skipped """
    elements_to_skip = len(eta_pair_idx_to_skip)

    configurations: List[dict] = []
    for eta_pair_idx in eta_pair_idx_to_skip:
        one_configuration = {'state_probability': 0,
                             'angle_rx1': 0,
                             'angle_ry1': 0,
                             'angle_rx0': 0,
                             'angle_ry0': 0,
                             'eta_pair': GLOBAL_ETA_PAIRS[eta_pair_idx]}
        configurations.append(one_configuration)

    return {'eta_pairs': [],
            'best_algorithm': ['NA'] * elements_to_skip,
            'probabilities': [0] * elements_to_skip,
            'configurations': configurations,
            'number_calls_made': [0] * elements_to_skip}

In [620]:
def _set_eta_pair_index_bounds(clone_setup: Optional[dict]) -> Tuple[int, int]:
    """ set the first and last eta pair index from which to optimize the configuration """
    total_eta_pairs = len(GLOBAL_ETA_PAIRS)

    if clone_setup is None or clone_setup['total_clones'] <= 1:
        return (0, total_eta_pairs)

    eta_pair_idx_init = int(np.floor(clone_setup['id_clone'] * total_eta_pairs / clone_setup['total_clones']))
    eta_pair_idx_end = min(int((clone_setup['id_clone'] + 1) *
                               total_eta_pairs / clone_setup['total_clones']), total_eta_pairs)
    return (eta_pair_idx_init, eta_pair_idx_end)

def _build_eta_pair_index_lists(eta_pair_idx_init: int, eta_pair_idx_end: int) -> Dict:
    """ create two lists with the the eta pair index to be computed and the index to be skipped """
    first_part_to_skip = list(range(0, eta_pair_idx_init))
    last_part_to_skip = list(range(eta_pair_idx_end, len(GLOBAL_ETA_PAIRS)))

    return {
        'eta_pair_idx_to_compute': list(range(eta_pair_idx_init, eta_pair_idx_end)),
        'eta_pair_idx_to_skip': first_part_to_skip + last_part_to_skip
    }

In [621]:
def _compute_best_configuration(optimization_setup: dict, GLOBAL_ETA_PAIR) -> dict:
    """ Find out the best configuration with a global pair of etas (channels) trying out
        a list of specified optimization algorithm """
    optimizer_algorithms = optimization_setup['optimizer_algorithms']
    optimizer_iterations = optimization_setup['optimizer_iterations']
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""
    number_calls_made = 0

    for optimizer_algorithm, max_evals in zip(optimizer_algorithms, optimizer_iterations):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'SLSQP':
            optimizer = SLSQP(maxiter=max_evals)
        if optimizer_algorithm == 'L_BFGS_B':
            optimizer = L_BFGS_B(maxfun=max_evals, maxiter=max_evals)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=max_evals)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=max_evals)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=max_evals)

        ret = optimizer.optimize(num_vars=len(optimization_setup['initial_parameters']),
                                 objective_function=_cost_function,
                                 variable_bounds=optimization_setup['variable_bounds'],
                                 initial_point=optimization_setup['initial_parameters'])
#        print('Parámetros')
#        print(ret[0])
        print("Best Average Probability:", -ret[1])
        if (-ret[1]) > best_probability:
            best_configuration = ret[0]
            best_probability = -ret[1]
            number_calls_made = ret[2]
            best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Number of cost function calls made:", number_calls_made)
    print("Parameters Found: state_probability = " + str(best_configuration[0]) +
          ", " + u"\u03D5" + "rx1 = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry1 = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" +
          ", " + u"\u03D5" + "rx0 = " + str(int(math.degrees(best_configuration[3]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry0 = " + str(int(math.degrees(best_configuration[4]))) + u"\u00B0" +
          ", " + u"\u03B7" + u"\u2080" + " = " + str(int(math.degrees(GLOBAL_ETA_PAIR[0]))) + u"\u00B0" +
          ", " + u"\u03B7" + u"\u2081" + " = " + str(int(math.degrees(GLOBAL_ETA_PAIR[1]))) + u"\u00B0")
    
    best_configuration_dict = {'state_probability': best_configuration[0],
                               'angle_rx1': best_configuration[1],
                               'angle_ry1': best_configuration[2],
                               'angle_rx0': best_configuration[3],
                               'angle_ry0': best_configuration[4],
                               'eta_pair': GLOBAL_ETA_PAIR} 
    
    return {'best_algorithm': best_optimizer_algorithm,
            'best_probability': best_probability,
            'best_configuration': best_configuration_dict,
            'number_calls_made': number_calls_made} 

In [622]:
def _cost_function(params: List[float]) -> float:
    """ Computes the cost of running a specific configuration for the number of plays
        defined in the optimization setup.
        Cost is computed as 1 (perfect probability) - average success probability for
        all the plays with the given configuration
        Returns the Cost (error probability).
    """
    configuration = {
        'state_probability': params[0],
        'angle_rx1': params[1],
        'angle_ry1': params[2],
        'angle_rx0': params[3],
        'angle_ry0': params[4],
        'eta_pair': GLOBAL_ETA_PAIR}

    return - compute_new_average_success_probability(configuration=configuration,
                                                       plays=GLOBAL_PLAYS)

## Auxiliary functions for fast compute the circuit and success average probability

In [623]:
def compute_new_average_success_probability(configuration: dict,
                                            plays: Optional[int] = 100,) -> float:
    """ Computes the average success probability of running a specific configuration
        for the number of plays defined in the configuration.
    """
    random_etas, eta_shots = _get_random_etas_and_eta_shots(plays)
    #guesses_eta = _run_all_circuits_and_return_guess(configuration, eta_shots)
    #return _check_guesses_and_return_average_success_probability(plays, random_etas, guesses_eta)
    return _run_all_circuits_and_return_cost(configuration, eta_shots)

In [624]:
def _get_random_etas_and_eta_shots(plays: int) -> Tuple[frozenbitarray, Tuple[int, int]]:
    """ create a random bit string with length the number of plays and calculate the
        number of shots for each eta based on the random string value (0 -> eta0, 1 -> eta1)
    """
    random_etas = frozenbitarray(urandom(plays))
    eta1_shots = random_etas.count()
    eta0_shots = plays - eta1_shots
    return (random_etas, (eta0_shots, eta1_shots))

In [625]:
def _check_guesses_and_return_average_success_probability(plays: int, 
                                                          random_etas: frozenbitarray, 
                                                          guesses_eta: Tuple[List[int],List[int]]) -> float:
    """ check the guessed etas with the initial random etas and compute the average with a xor bitwise operation """
    guesses = frozenbitarray([guesses_eta[random_eta].pop(0) for random_eta in random_etas])
    return 1 - (count_xor(random_etas, guesses) / plays)

In [626]:
def _run_all_circuits_and_return_cost(configuration: dict,
                                       eta_shots: Optional[Tuple[int, int]] = (1, 1)) -> Tuple[List[int],
                                                                                               List[int]]:
    """ Create a pair of Quantum Circuits, in its transpiled form, from a given configuration """
    if eta_shots is None:
        eta_shots = (1, 1)

    reordered_configuration = _reorder_configuration(configuration)
    eta_counts = [cast(Result, execute(_create_one_circuit(reordered_configuration, eta),
                                         backend=GLOBAL_BACKEND,
                                         shots=eta_shots[idx],
                                         memory=False).result()).get_counts()
                    for idx, eta in enumerate(reordered_configuration['eta_pair'])]
    
    counts_distribution=[0.0, 0.0, 0.0, 0.0]
    if '00' in eta_counts[0]: 
        counts_distribution[0]= eta_counts[0]['00']
    if '01' in eta_counts[0]: 
        counts_distribution[1]= eta_counts[0]['01']
    if '10' in eta_counts[0]: 
        counts_distribution[2]= eta_counts[0]['10']
    if '11' in eta_counts[0]: 
        counts_distribution[3]= eta_counts[0]['11']
    if '00' in eta_counts[1]: 
        counts_distribution[0]= max([counts_distribution[0], eta_counts[1]['00']])
    if '01' in eta_counts[1]: 
        counts_distribution[1]= max([counts_distribution[1], eta_counts[1]['01']])
    if '10' in eta_counts[1]: 
        counts_distribution[2]= max([counts_distribution[2], eta_counts[1]['10']])
    if '11' in eta_counts[1]: 
        counts_distribution[3]= max([counts_distribution[3], eta_counts[1]['11']])
#    print(counts_distribution)
#    print(configuration)

    cost= (counts_distribution[0] + counts_distribution[1] + counts_distribution[2] + counts_distribution[3])/GLOBAL_PLAYS
#    print(cost)
    return cost

In [627]:
def _reorder_configuration(configuration: dict):
    return {'state_probability': configuration['state_probability'],
            'angle_rx1': configuration['angle_rx1'],
            'angle_ry1': configuration['angle_ry1'],
            'angle_rx0': configuration['angle_rx0'],
            'angle_ry0': configuration['angle_ry0'],
            'eta_pair': reorder_pair(configuration['eta_pair'])
    }

In [628]:
def _create_one_circuit(configuration: dict,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given configuration and eta """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration['state_probability'])

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.barrier()
    circuit.rx(configuration['angle_rx1'], qreg_q[1])
    circuit.ry(configuration['angle_ry1'], qreg_q[1])
    circuit.rx(configuration['angle_rx0'], qreg_q[0])
    circuit.ry(configuration['angle_ry0'], qreg_q[0])
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

In [629]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    # ORIGINAL
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)
    # return (0, np.sqrt(1 - state_probability), np.sqrt(state_probability), 0)

In [630]:
def _get_counts_from_one_eta_counts(one_eta_counts: List[str]) -> List[int]:
    return [_guess_eta_from_counts(one_eta_count) for one_eta_count in one_eta_counts]

In [631]:
def _guess_eta_from_counts(counts: str) -> int:
    """ Decides which eta was used on the real execution from the 'counts' measured
        based on the guess strategy that is required to use
    """
    return _guess_eta_used_two_bit_strategy(counts)

In [632]:
def _guess_eta_used_two_bit_strategy(counts: str) -> int:
    """ Decides which eta was used on the real execution from the two 'counts' measured
        Qubits order MATTER!!!!
        "01" means that:
          the LEFTMOST bit (0) corresponds to the measurement of the qubit that goes THROUGH the channel
          and the RIGHTMOST bit (1) corresponds to the measurement of the qubit that goes OUTSIDE the channel
        Remember that we are only sending |01> + |10> entangles states
        Setting eta0 >= eta1:
            * outcome 00 -> eta0 as the most probable (more attenuation)
            * outcome 01 -> we do not know if there has been attenuation. 50% chance, random choice
            * outcome 10 -> eta1 as the most probable (less attenuation)
            * outcome 11 -> not possible, but in case we get it (from noisy simulation), 50% chance, random choice
    """
    if len(counts) != 2:
        raise ValueError('counts MUST be a two character length string')
    if counts == "00":
        return 0
    if counts == "10":
        return 1
    if counts == "01" or counts == "11":
        return random.choice([0, 1])
    raise ValueError("Accepted counts are '00', '01', '10', '11'")

## Launch Optimization

In [ ]:
clone_setup=None
eta_pairs_idx_to_optimize, optimal_configurations = _select_eta_pairs_to_optimize(clone_setup)

print(f'number of eta_pairs_idx_to_optimize: {len(eta_pairs_idx_to_optimize)} -> {eta_pairs_idx_to_optimize}')

program_start_time = time.time()
print("Starting the execution")
    
for eta_pair_idx in eta_pairs_idx_to_optimize:
    start_time = time.time()
    GLOBAL_ETA_PAIR = GLOBAL_ETA_PAIRS[eta_pair_idx]
    result = _compute_best_configuration(optimization_setup, GLOBAL_ETA_PAIR)
    # print(result)
    optimal_configurations['probabilities'].append(result['best_probability'])
    optimal_configurations['configurations'].append(result['best_configuration'])
    optimal_configurations['best_algorithm'].append(result['best_algorithm'])
    optimal_configurations['number_calls_made'].append(result['number_calls_made'])
    end_time = time.time()
    print(f"Pair of etas # {eta_pair_idx+1} of {len(eta_pairs_idx_to_optimize)}, time taken this pair of etas: " +
              f'{np.round((end_time - start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - start_time) % 60, 0)} seconds')
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time) / 60)))
    print("total time taken so far: " +
              f'{np.round((end_time - program_start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - program_start_time) % 60, 0)} seconds')
end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time) / 60)))
print(f'Number eta pairs optimized: {len(eta_pairs_idx_to_optimize)}' +
          f'from the total eta pairs: {len(GLOBAL_ETA_PAIRS)} ')
optimal_configurations['eta_pairs'] = GLOBAL_ETA_PAIRS
results= optimal_configurations

number of eta_pairs_idx_to_optimize: 210 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
Starting the ex

Best Average Probability: 0.9573
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9573
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.0006858710562414549, ϕrx1 = 89°, ϕry1 = 108°, ϕrx0 = 89°, ϕry0 = 92°, η₀ = 72°, η₁ = 0°
Pair of etas # 16 of 210, time taken this pair of etas: 1.0 minutes and 48.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 12.0 minutes and 31.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.976
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.976
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.0020576131687243078, ϕrx1 = 90°, ϕry1 = 10°, ϕrx0 = 90°, ϕry0 = 67°, η₀ = 76°, η₁ = 0°
Pair of etas # 17 of 210, time taken this pair of etas: 1.0 minutes and 34.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 12.0 minutes and 5.0 seconds
Analyzing Optimizer Algorithm:  D

Best Average Probability: 0.9006
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9006
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 87°, ϕry1 = 130°, ϕrx0 = 90°, ϕry0 = 96°, η₀ = 63°, η₁ = 4°
Pair of etas # 33 of 210, time taken this pair of etas: 1.0 minutes and 36.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 23.0 minutes and 6.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.9281
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9281
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 92°, ϕry1 = 34°, ϕrx0 = 87°, ϕry0 = 167°, η₀ = 67°, η₁ = 4°
Pair of etas # 34 of 210, time taken this pair of etas: 1.0 minutes and 48.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 24.0 minutes and 54.0 seconds
Analyzing Optimizer Algorithm:  

Best Average Probability: 0.8581
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8581
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 90°, ϕry1 = 70°, ϕrx0 = 95°, ϕry0 = 83°, η₀ = 58°, η₁ = 9°
Pair of etas # 50 of 210, time taken this pair of etas: 1.0 minutes and 37.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 35.0 minutes and 1.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.8914
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8914
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 87°, ϕry1 = 12°, ϕrx0 = 90°, ϕry0 = 70°, η₀ = 63°, η₁ = 9°
Pair of etas # 51 of 210, time taken this pair of etas: 1.0 minutes and 38.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 36.0 minutes and 39.0 seconds
Analyzing Optimizer Algorithm:  DI

Best Average Probability: 0.8441
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8441
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 90°, ϕry1 = 143°, ϕrx0 = 89°, ϕry0 = 149°, η₀ = 58°, η₁ = 13°
Pair of etas # 67 of 210, time taken this pair of etas: 1.0 minutes and 46.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 47.0 minutes and 41.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.8779
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8779
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.004801097393690014, ϕrx1 = 89°, ϕry1 = 167°, ϕrx0 = 87°, ϕry0 = 143°, η₀ = 63°, η₁ = 13°
Pair of etas # 68 of 210, time taken this pair of etas: 1.0 minutes and 38.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 47.0 minutes and 19.0 seconds
Analyzing Optimizer Algorit

Best Average Probability: 0.8594
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8594
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.0020576128819271085, ϕrx1 = 89°, ϕry1 = 130°, ϕrx0 = 89°, ϕry0 = 87°, η₀ = 63°, η₁ = 18°
Pair of etas # 84 of 210, time taken this pair of etas: 1.0 minutes and 38.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 58.0 minutes and 30.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.8856
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8856
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 92°, ϕry1 = 161°, ϕrx0 = 90°, ϕry0 = 110°, η₀ = 67°, η₁ = 18°
Pair of etas # 85 of 210, time taken this pair of etas: 1.0 minutes and 47.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 59.0 minutes and 17.0 seconds
Analyzing Optimizer Algorit

Best Average Probability: 0.8846
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.8846
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 90°, ϕry1 = 109°, ϕrx0 = 87°, ϕry0 = 105°, η₀ = 72°, η₁ = 22°
Pair of etas # 101 of 210, time taken this pair of etas: 1.0 minutes and 40.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 70.0 minutes and 27.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.9063
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9063
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.0020576131687243078, ϕrx1 = 94°, ϕry1 = 83°, ϕrx0 = 90°, ϕry0 = 70°, η₀ = 76°, η₁ = 22°
Pair of etas # 102 of 210, time taken this pair of etas: 1.0 minutes and 36.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 71.0 minutes and 4.0 seconds
Analyzing Optimizer Algorit

Best Average Probability: 0.9003
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9003
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 87°, ϕry1 = 81°, ϕrx0 = 89°, ϕry0 = 104°, η₀ = 85°, η₁ = 27°
Pair of etas # 118 of 210, time taken this pair of etas: 1.0 minutes and 43.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 82.0 minutes and 8.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.9039
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.9039
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006170957829749708, ϕrx1 = 90°, ϕry1 = 25°, ϕrx0 = 87°, ϕry0 = 7°, η₀ = 90°, η₁ = 27°
Pair of etas # 119 of 210, time taken this pair of etas: 1.0 minutes and 41.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 83.0 minutes and 49.0 seconds
Analyzing Optimizer Algorithm:

Best Average Probability: 0.6313
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.6313
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.006172839506172867, ϕrx1 = 90°, ϕry1 = 3°, ϕrx0 = 92°, ϕry0 = 110°, η₀ = 49°, η₁ = 36°
Pair of etas # 135 of 210, time taken this pair of etas: 1.0 minutes and 38.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 94.0 minutes and 54.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.665
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.665
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.018518518518518545, ϕrx1 = 83°, ϕry1 = 110°, ϕrx0 = 90°, ϕry0 = 72°, η₀ = 54°, η₁ = 36°
Pair of etas # 136 of 210, time taken this pair of etas: 1.0 minutes and 50.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 95.0 minutes and 44.0 seconds
Analyzing Optimizer Algorithm:

Best Average Probability: 0.7697
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.7697
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.01806127114769093, ϕrx1 = 83°, ϕry1 = 50°, ϕrx0 = 90°, ϕry0 = 123°, η₀ = 76°, η₁ = 40°
Pair of etas # 152 of 210, time taken this pair of etas: 1.0 minutes and 50.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 108.0 minutes and 48.0 seconds
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.7839
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.7839
Number of cost function calls made: 1000
Parameters Found: state_probability = 0.018518518518518545, ϕrx1 = 90°, ϕry1 = 50°, ϕrx0 = 90°, ϕry0 = 101°, η₀ = 81°, η₁ = 40°
Pair of etas # 153 of 210, time taken this pair of etas: 1.0 minutes and 48.0 seconds
total minutes taken this pair of etas:  1
total time taken so far: 109.0 minutes and 36.0 seconds
Analyzing Optimizer Algori

In [ ]:
# results

In [ ]:
save_results_to_disk(results, filename)
# save_results_to_disk(results, '20210412d_C2_A1_100_10000')

In [ ]:
qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)
circuit.initialize([0,1,0,0], [0, 1])
circuit.reset(qreg_q[2])
circuit.cry(0, qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[2], qreg_q[1])
circuit.barrier()
circuit.rx(0, qreg_q[1])
circuit.ry(0, qreg_q[1])
circuit.rx(0, qreg_q[0])
circuit.ry(0, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.measure([0, 1], creg_c)
circuit.draw()